In [2]:
######################################     Modules     #######################################
import sys
from time import process_time
import os
import pickle

sys.path.append('../.')
from verbose_module import routing_instances as verb
sys.path.append('../../../.')
import pIRPgym


computer_name = input("Running experiment on mac? [Y/n]")
if computer_name == '': 
    path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
    experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/'
else: 
    path = 'C:/Users/jm.betancourt/Documents/Research/pIRPgym/'
    experiments_path = 'G:/Mi unidad/Research/Supply Chain Analytics/Experiments/Classic Instances/'


instances = dict()
instances['Li'] = [i for i in os.listdir(path+'/pIRPgym/Instances/CVRP Instances/dCVRP/Li') if i[-3:]=='vrp']
instances['Golden'] = [i for i in os.listdir(path+'/pIRPgym/Instances/CVRP Instances/dCVRP/Golden') if i[-3:]=='vrp']
instances['Uchoa'] = [i for i in os.listdir(path+'pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
instances['Li'].sort();instances['Golden'].sort()
instances['Uchoa'].sort();instances['Uchoa'] = instances['Uchoa'][1:]+[instances['Uchoa'][0]]

Policies = ['NN','RCL']

verbose = True

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


# dCVRP Instances

In [7]:
verbose = True

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)

for inst_set in ['Li','Golden']:
    if verbose: verb.print_head(Policies,inst_set,show_gap=True)
    
    for instance in instances[inst_set][]:
        purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
        if verbose: string = verb.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
        
        for policy in Policies:
            end=False
            if policy==Policies[-1]: end=True
            # print(f'{inst_set}/{policy}/{instance[:-4]}.pkl')
            with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
            
            if policy=='NN':
                if verbose: string = verb.print_routing_update(string,performance[1],len(performance[0]),
                                                                                 performance[3],True,benchmark,end=end)
            elif policy=='RCL':
                if verbose: string = verb.print_routing_update(string,performance[0],performance[1],
                                                                                 performance[2],True,benchmark,intervals=performance[3],end=end)
    
    print('\n')

[29162.5, 14.6, 4.815625, (7170.1186914862155, 22736, 45004)]
[31612.8, 14.6, 4.9, (5776.34958775869, 26874, 43366)]
[23785.0, 30.4, 4.821875, (1571.8404499185024, 22907, 28456)]
[35745.8, 15.5, 4.9359375, (7888.948938863782, 30106, 52560)]
[25504.1, 32.5, 4.9234375, (2532.152896252515, 23463, 30378)]
[35068.2, 13.9, 5.0328125, (1483.6456315441367, 33276, 37705)]
[38339.7, 14.0, 5.44375, (1308.8168741271638, 35603, 40215)]
[48856.1, 16.4, 5.2421875, (13027.049584998133, 41142, 75130)]
[46392.5, 14.5, 5.534375, (1402.6488691044526, 44546, 48139)]
[50347.1, 15.1, 5.7609375, (2399.397776526435, 46674, 53941)]


[15713.9, 14.8, 4.5515625, (1508.4965661213816, 14222, 19201)]
[17933.0, 13.3, 4.571875, (623.5310738046661, 16777, 19032)]
[8475.4, 5.7, 4.5125, (495.9488280054708, 7712, 9510)]
[11005.2, 8.7, 4.54375, (932.4252034345704, 10047, 13310)]
[13680.3, 12.3, 4.5484375, (1070.8271615905155, 12918, 16619)]
[16683.7, 16.0, 4.5484375, (1270.469680866096, 15577, 19445)]




### GA 

In [ ]:
POPULATION_SIZES = [250,750,1000,2500]
ELITE_PROPORTIONS = [0.05,0.15,0.3]
MUTATION_RATES = [0.25,0.5,0.75]

combinations = [(p,e,m) for p in POPULATION_SIZES for e in ELITE_PROPORTIONS for m in MUTATION_RATES]


# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)


for instance in [instances['Li'][4]]:
    purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)

    for (p,e,m) in combinations:
        with open(experiments_path+f'{inst_set}/GA/{instance[:-4]}_{p}_{e}_{m}.pkl', 'rb') as file:
            # Use pickle.dump to serialize and save the dictionary to the file
            performance = pickle.load(file)

            # print(f'{p}_{e}_{m} \t',round((performance[1]-benchmark[0])/benchmark[0],2))
            print(f'{p}_{e}_{m} \t',performance[-1])
            

# CVRP Instances

### exo-alpha

In [5]:
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.routing_instances.print_head(Policies,inst_set,show_gap=True)

for instance in instances[inst_set][1:]:
    purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
    if verbose: string = verb.routing_instances.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
    
    for policy in Policies:
        end=False
        if policy==Policies[-1]: end=True

        if policy == 'RCL':
            with open(experiments_path+f'{inst_set}/{policy}/Random/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        else:
            with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        
        if policy=='NN':
            if verbose: string = verb.routing_instances.print_routing_update(string,performance[1],len(performance[0]),
                                                                                performance[3],True,benchmark,end=end)
        elif policy=='RCL':
            if verbose: 
                string = verb.routing_instances.print_routing_update(string,performance[0],performance[1],
                                                                                performance[2],True,benchmark,intervals=performance[3],end=end)
            RCL_averge_gap+=((performance[0]-benchmark[0])/benchmark[0])/100

print('RCL average gap:', round(RCL_averge_gap,4)*100)

AttributeError: type object 'routing_instances' has no attribute 'routing_instances'

In [6]:
def create_node_range_library(instance_names):
    node_range_library = {}

    for instance_name in instance_names:
        # Extract the number of nodes from the instance name
        nodes = int(instance_name.split('-n')[1].split('-')[0])

        # Determine the range tuple
        node_range = ((nodes // 100) * 100, ((nodes // 100) * 100) + 99)

        # Add the instance to the corresponding range in the dictionary
        if node_range not in node_range_library:
            node_range_library[node_range] = [instance_name]
        else:
            node_range_library[node_range].append(instance_name)

    return node_range_library

# Example usage:
# Call the function to create the library
instance_sizes = create_node_range_library(instances['Uchoa'])

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_comparison_head(Policies,inst_set,show_gap=True)

for interval,inst_list in instance_sizes.items():

    if verbose: string = verb.print_comparison_inst(interval,len(inst_list))
    NN_gap = 0
    NN_time = 0
    RCL_gap = 0
    RCL_time = 0
    RCL_min = 0
    RCL_max = 0

    for instance in inst_list:
        purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

        with open(experiments_path+f'Uchoa/NN/{instance[:-4]}.pkl', 'rb') as file:
            NN_performance = pickle.load(file)
            NN_gap += (((NN_performance[1]-benchmark[0])/benchmark[0])/len(inst_list))
            NN_time += NN_performance[3]/len(inst_set)


        with open(experiments_path+f'Uchoa/RCL/Random/{instance[:-4]}.pkl', 'rb') as file:
            RCL_performance = pickle.load(file)
            RCL_gap += ((RCL_performance[0]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_time += RCL_performance[2]/len(inst_list)/len(inst_list)
            RCL_min += ((RCL_performance[3][1]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_max += ((RCL_performance[3][2]-benchmark[0])/benchmark[0])/len(inst_list)
    

    verb.print_routing_comparison_update(string,NN_gap,NN_time,RCL_gap,RCL_time,RCL_min,RCL_max)


****************************  Uchoa set Instances  *****************************
----------------|	NN 	|	     RCL 	 	|
Sizes	  Num 	| t(s) 	   gap 	| t(s)	  mean	 median 	   min	   max	|
-----------------------------------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/Uchoa/RCL/Random/X-n101-k25.pkl'

### intra-alpha

In [ ]:
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_head(Policies,inst_set,show_gap=True)

for instance in instances[inst_set][1:]:
    purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
    if verbose: string = verb.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
    
    for policy in Policies:
        end=False
        if policy==Policies[-1]: end=True

        if policy == 'RCL':
            with open(experiments_path+f'{inst_set}/{policy}/Adaptative/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        else:
            with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        
        if policy=='NN':
            if verbose: string = verb.print_routing_update(string,performance[1],len(performance[0]),
                                                                                performance[3],True,benchmark,end=end)
        elif policy=='RCL':
            if verbose: string = verb.print_routing_update(string,performance[0],performance[1],
                                                                                performance[2],True,benchmark,intervals=performance[3],end=end)
            RCL_averge_gap+=((performance[0]-benchmark[0])/benchmark[0])/100

print('RCL average gap', round(RCL_averge_gap,4)*100)

In [ ]:
def create_node_range_library(instance_names):
    node_range_library = {}

    for instance_name in instance_names:
        # Extract the number of nodes from the instance name
        nodes = int(instance_name.split('-n')[1].split('-')[0])

        # Determine the range tuple
        node_range = ((nodes // 100) * 100, ((nodes // 100) * 100) + 99)

        # Add the instance to the corresponding range in the dictionary
        if node_range not in node_range_library:
            node_range_library[node_range] = [instance_name]
        else:
            node_range_library[node_range].append(instance_name)

    return node_range_library

# Example usage:
# Call the function to create the library
instance_sizes = create_node_range_library(instances['Uchoa'])

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_comparison_head(Policies,inst_set,show_gap=True)

for interval,inst_list in instance_sizes.items():

    if verbose: string = verb.print_comparison_inst(interval,len(inst_list))
    NN_gap = 0
    NN_time = 0
    RCL_gap = 0
    RCL_time = 0
    RCL_min = 0
    RCL_max = 0

    for instance in inst_list:
        purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

        with open(experiments_path+f'Uchoa/NN/{instance[:-4]}.pkl', 'rb') as file:
            NN_performance = pickle.load(file)
            NN_gap += (((NN_performance[1]-benchmark[0])/benchmark[0])/len(inst_list))
            NN_time += NN_performance[3]/len(inst_set)


        with open(experiments_path+f'Uchoa/RCL/Adaptative/{instance[:-4]}.pkl', 'rb') as file:
            RCL_performance = pickle.load(file)
            RCL_gap += ((RCL_performance[0]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_time += RCL_performance[2]/len(inst_list)/len(inst_list)
            RCL_min += ((RCL_performance[3][1]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_max += ((RCL_performance[3][2]-benchmark[0])/benchmark[0])/len(inst_list)
    

    verb.print_routing_comparison_update(string,NN_gap,NN_time,RCL_gap,RCL_time,RCL_min,RCL_max)
